# Elo Score Calculation

In [1]:
import re
from collections import defaultdict

In [2]:
import numpy as np
import pandas as pd

## Summary

- Notebook that calcuates the Elo Score of mice competing over access to a tone associated reward port
- The data is a spreadsheet of combination of subjects (as rows) against dates (as columns)

## Importing Data

- The original data has been trimmed so that only the cells with dates, winner ID's, and relevant metadata were kept

In [3]:
tube_test_df = pd.read_csv("../../data/tube_test_fights.csv")

In [4]:
tube_test_df.head()

,cage,animal,25-Apr,26-Apr,27-Apr,28-Apr,29-Apr,2-May,4-May,5-May,6-May,9-May,10-May,11-May,12-May,13-May,20-May
0,1,1.1 v 2.2,1.1,1.1,1.1,2.2,1.1,1.1,NaN,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1
1,1,2.2 v 2.3,2.2,2.3,2.2,2.2,2.2,2.2,NaN,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2
2,1,2.3 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
3,1,1.4 v 1.1,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
4,1,2.2 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4


## Doing it for a subset of the data

- To test out the code, we will be using the data from only one cage

In [5]:
# Getting all the rows that are from cage 1
cage_1_df = tube_test_df[tube_test_df["cage"] == 1]

In [6]:
cage_1_df

,cage,animal,25-Apr,26-Apr,27-Apr,28-Apr,29-Apr,2-May,4-May,5-May,6-May,9-May,10-May,11-May,12-May,13-May,20-May
0,1,1.1 v 2.2,1.1,1.1,1.1,2.2,1.1,1.1,NaN,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1
1,1,2.2 v 2.3,2.2,2.3,2.2,2.2,2.2,2.2,NaN,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2
2,1,2.3 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
3,1,1.4 v 1.1,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
4,1,2.2 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,NaN,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
5,1,1.1 v 2.3,2.3,1.1,1.1,1.1,1.1,1.1,NaN,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1


## Getting a list of all the animals

- Because each cell does not contain the ID of all the animals that were competing, we will make a new column with both ID's

In [7]:
def get_all_animal_ids(animal_string):
    """
    Converts a string that contains the ID of animals, and only gets the IDs. 
    This usually removes extra characters that were added. (i.e. "1.1 v 2.2" to ("1.1", "2.2"))

    Args:
        animal_string(str): This is the first param.

    Returns:
        tuple: Of IDs of animals as strings
    """
    # Splitting by space so that we have a list of just the words
    all_words = animal_string.split()
    # Removing all words that are not numbers
    all_numbers = [num for num in all_words if re.match(r'^-?\d+(?:\.\d+)$', num)]
    return tuple(all_numbers)


In [8]:
print(get_all_animal_ids("1.1 v 2.3"))

('1.1', '2.3')


- Turning all the columns into string so that we can match the ID's from one column to another

In [9]:
for col in cage_1_df.columns:
    cage_1_df[col] = cage_1_df[col].astype(str)

/tmp/ipykernel_140026/4260518769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cage_1_df[col] = cage_1_df[col].astype(str)


In [10]:
cage_1_df.head()

,cage,animal,25-Apr,26-Apr,27-Apr,28-Apr,29-Apr,2-May,4-May,5-May,6-May,9-May,10-May,11-May,12-May,13-May,20-May
0,1,1.1 v 2.2,1.1,1.1,1.1,2.2,1.1,1.1,nan,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1
1,1,2.2 v 2.3,2.2,2.3,2.2,2.2,2.2,2.2,nan,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2
2,1,2.3 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
3,1,1.4 v 1.1,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
4,1,2.2 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4


In [11]:
cage_1_df["all_animals"] = cage_1_df["animal"].apply(lambda x: get_all_animal_ids(x))

/tmp/ipykernel_140026/1018194899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cage_1_df["all_animals"] = cage_1_df["animal"].apply(lambda x: get_all_animal_ids(x))


In [12]:
cage_1_df.head()

,cage,animal,25-Apr,26-Apr,27-Apr,28-Apr,29-Apr,2-May,4-May,5-May,6-May,9-May,10-May,11-May,12-May,13-May,20-May,all_animals
0,1,1.1 v 2.2,1.1,1.1,1.1,2.2,1.1,1.1,nan,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,"(1.1, 2.2)"
1,1,2.2 v 2.3,2.2,2.3,2.2,2.2,2.2,2.2,nan,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,"(2.2, 2.3)"
2,1,2.3 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,"(2.3, 1.4)"
3,1,1.4 v 1.1,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,"(1.4, 1.1)"
4,1,2.2 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,"(2.2, 1.4)"


## Elo Score Calculation

In [13]:
def calculate_elo_score(subject_elo_score, agent_elo_score, k_factor=20, score=1, number_of_decimals=None):
    """
    Calculates the Elo score of a given subject given it's original score, it's opponent, 
    the K-Factor, and whether or not it has won or not. 
    The calculation is based on: https://www.omnicalculator.com/sports/elo

    Args:
        subject_elo_score(float): The original Elo score for the subject
        agent_elo_score(float): The original Elo score for the agent
        k_factor(int): k-factor, or development coefficient. 
            - It usually takes values between 10 and 40, depending on player's strength 
        score(int): the actual outcome of the game. 
            - In chess, a win counts as 1 point, a draw is equal to 0.5, and a lose gives 0.
        number_of_decimals(int): Number of decimals to round to
        
    Returns:
        int: Updated Elo score of the subject
    """
    # Calculating the Elo score
    rating_difference = agent_elo_score - subject_elo_score
    expected_score = 1 / (1 + 10 ** (rating_difference / 400))
    new_elo_score = subject_elo_score + k_factor * (score - expected_score)
    # Rounding to `number_of_decimals`
    return round(new_elo_score, number_of_decimals)

In [14]:
calculate_elo_score(subject_elo_score=1500, agent_elo_score=500, score=0)

1480

## Calculate all the Elo scores for this cage

In [15]:
cage_1_df.columns

Index(['cage', 'animal', '25-Apr', '26-Apr', '27-Apr', '28-Apr', '29-Apr',
       '2-May', '4-May', '5-May', '6-May', '9-May', '10-May', '11-May',
       '12-May', '13-May', '20-May', 'all_animals'],
      dtype='object')

In [16]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))
for column in cage_1_df:
    if column not in ['cage', 'animal', 'all_animals']:
        print("Current Date: {}".format(column))
        # Keeping track of the number of matches
        id_to_match_number = defaultdict(lambda:1)
        for index, row in cage_1_df.iterrows():
            # Checking if there is an Nan or not
            if row[column] == "nan":
                continue
            else:            
                winner_id = row[column]

            # Getting the ID of the loser subject
            loser_id = list(set(row["all_animals"]) - set([winner_id]))
            loser_id = loser_id[0]
            # Getting the current Elo Score
            current_winner_rating = id_to_elo_score[winner_id] 
            current_loser_rating = id_to_elo_score[loser_id] 
            # Calculating Elo score            
            id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=1)
            id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=0)
            
            # Saving all the data for the winner
            winner_index = next(all_indexes)
            index_to_elo_score_and_meta_data[winner_index]["date"] = column
            index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[winner_id]
            index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
            index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
            index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
            index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
            index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = 1
            
            # Saving all the data for the loser
            loser_index = next(all_indexes)
            index_to_elo_score_and_meta_data[loser_index]["date"] = column
            index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[loser_id]
            index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
            index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
            index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
            index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
            index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = 0
            
            id_to_match_number[winner_id] += 1
            id_to_match_number[loser_id] += 1
        break
    

Current Date: 25-Apr


# Calculate Elo score for all cells

In [17]:
for col in tube_test_df.columns:
    tube_test_df[col] = tube_test_df[col].astype(str)

In [18]:
rename_dates_dict = {"25-Apr": "04_25",  "26-Apr": "04_26",  "27-Apr": "04_27",  "28-Apr": "04_28",  "29-Apr": "04_29",  "2-May": "05_02", "4-May": "05_04", "5-May": "05_05", "6-May": "05_06", "9-May": "05_09", "10-May": "05_10", "11-May": "05_11", "12-May": "05_12", "13-May": "05_13",  "20-May": "05_20"}

In [19]:
tube_test_df = tube_test_df.rename(columns=rename_dates_dict)

In [20]:
tube_test_df.head()

,cage,animal,04_25,04_26,04_27,04_28,04_29,05_02,05_04,05_05,05_06,05_09,05_10,05_11,05_12,05_13,05_20
0,1,1.1 v 2.2,1.1,1.1,1.1,2.2,1.1,1.1,nan,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1
1,1,2.2 v 2.3,2.2,2.3,2.2,2.2,2.2,2.2,nan,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2
2,1,2.3 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
3,1,1.4 v 1.1,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
4,1,2.2 v 1.4,1.4,1.4,1.4,1.4,1.4,1.4,nan,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4


In [21]:
tube_test_df["all_animals"] = tube_test_df["animal"].apply(lambda x: get_all_animal_ids(x))

## Get the Elo score for one fight

In [22]:
tube_test_df.columns

Index(['cage', 'animal', '04_25', '04_26', '04_27', '04_28', '04_29', '05_02',
       '05_04', '05_05', '05_06', '05_09', '05_10', '05_11', '05_12', '05_13',
       '05_20', 'all_animals'],
      dtype='object')

In [23]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))
for column in tube_test_df:
    if column not in ['cage', 'animal', 'all_animals']:
        print("Current Date: {}".format(column))
        # Keeping track of the number of matches
        id_to_match_number = defaultdict(lambda:1)
        for index, row in tube_test_df.iterrows():
            # Checking if there is an Nan or not
            if row[column] == "nan":
                continue
            else:            
                winner_id = row[column]

            # Getting the ID of the loser subject
            loser_id = list(set(row["all_animals"]) - set([winner_id]))
            loser_id = loser_id[0]
            # Getting the current Elo Score
            current_winner_rating = id_to_elo_score[winner_id] 
            current_loser_rating = id_to_elo_score[loser_id] 
            # Calculating Elo score            
            id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=1, number_of_decimals=1)
            id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=0, number_of_decimals=1)
            
            # Saving all the data for the winner
            winner_index = next(all_indexes)
            index_to_elo_score_and_meta_data[winner_index]["date"] = column
            index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[winner_id]
            index_to_elo_score_and_meta_data[winner_index]["cage"] = row["cage"]
            index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
            index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
            index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
            index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
            index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = 1
            
            
            # Saving all the data for the loser
            loser_index = next(all_indexes)
            index_to_elo_score_and_meta_data[loser_index]["date"] = column
            index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[loser_id]
            index_to_elo_score_and_meta_data[loser_index]["cage"] = row["cage"]
            index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
            index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
            index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
            index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
            index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = 0
            
            id_to_match_number[winner_id] += 1
            id_to_match_number[loser_id] += 1
    

Current Date: 04_25
Current Date: 04_26
Current Date: 04_27
Current Date: 04_28
Current Date: 04_29
Current Date: 05_02
Current Date: 05_04
Current Date: 05_05
Current Date: 05_06
Current Date: 05_09
Current Date: 05_10
Current Date: 05_11
Current Date: 05_12
Current Date: 05_13
Current Date: 05_20


In [24]:
id_to_elo_score

defaultdict(<function __main__.<lambda>()>,
            {'1.1': 1065.7,
             '2.2': 937.8,
             '2.3': 778.9,
             '1.4': 1238.0,
             '1.2': 937.4,
             '1.3': 781.4,
             '2.4': 1145.0,
             '2.1': 1115.8,
             '3.2': 1228.3,
             '4.2': 1082.2,
             '3.3': 905.0,
             '4.1': 784.5,
             '4.3': 1110.6,
             '3.4': 1129.5,
             '3.1': 981.9,
             '4.4': 778.0})

In [25]:
index_to_elo_score_and_meta_data[0]

{'date': '04_25',
 'match_number': 1,
 'cage': '1',
 'subject_id': '1.1',
 'agent_id': '2.2',
 'original_elo_score': 1000,
 'updated_elo_score': 1010.0,
 'win_draw_loss': 1}

In [26]:
elo_score_df = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

In [27]:
elo_score_df.head(n=25)

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,04_25,1,1,1.1,2.2,1000.0,1010.0,1
1,04_25,1,1,2.2,1.1,1000.0,990.0,0
2,04_25,2,1,2.2,2.3,990.0,1000.3,1
3,04_25,1,1,2.3,2.2,1000.0,989.7,0
4,04_25,1,1,1.4,2.3,1000.0,1009.7,1
5,04_25,2,1,2.3,1.4,989.7,980.0,0
6,04_25,2,1,1.4,1.1,1009.7,1019.7,1
7,04_25,2,1,1.1,1.4,1010.0,1000.0,0
8,04_25,3,1,1.4,2.2,1019.7,1029.1,1
9,04_25,3,1,2.2,1.4,1000.3,990.9,0


In [28]:
elo_score_df.to_csv("./proc/tube_test_elo_score.csv")
# elo_score_df.to_excel("./proc/id_to_date_elo_score.xlsx")

# DROP days that have have draws

# Calculate Elo score for Home Cage Observations

In [29]:
home_cage_observations_df = pd.read_excel("../../data/Homecage observations.xlsx", sheet_name="Master List")

In [30]:
home_cage_observations_df["date_str"] = home_cage_observations_df["Unnamed: 0"].astype(str)

In [31]:
home_cage_observations_df.head()

,Unnamed: 0,Match,Winner,date_str
0,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02
1,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02
2,2022-05-02,4.4 (4) v 3.1 (1),4.4 (4),2022-05-02
3,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02
4,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02


In [32]:
home_cage_observations_df["winner_str"] = home_cage_observations_df["Winner "].apply(lambda x: x.strip())

# Add Cage information

In [33]:
id_to_cage = {"1.1": "1",
"2.2": "1",
"2.3": "1",
"1.4": "1",

"2.1": "2",
"1.2": "2",
"1.3": "2", 
"2.4": "2",

"4.1 (1)": "3", 
"3.2 (2)": "3", 
"4.2 (3)": "3", 
"3.3 (4)": "3", 

"3.1 (1)": "4",
"3.4 (2)": "4", 
"4.3 (3)": "4",
"4.4 (4)": "4"}

In [34]:
home_cage_observations_df["cage"] = home_cage_observations_df["winner_str"].map(id_to_cage)

In [35]:
home_cage_observations_df.head()

,Unnamed: 0,Match,Winner,date_str,winner_str,cage
0,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4
1,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02,4.4 (4),4
2,2022-05-02,4.4 (4) v 3.1 (1),4.4 (4),2022-05-02,4.4 (4),4
3,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02,4.4 (4),4
4,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4


# Calculate Elo score for all cells

- Getting all the ID's of the animals

In [36]:
# Splitting by "v" and stripping all the spaces
home_cage_observations_df["all_animals"] = home_cage_observations_df["Match"].apply(lambda x: tuple([animal.strip() for animal in x.split("v")]))

In [37]:
home_cage_observations_df.head(n=25)

,Unnamed: 0,Match,Winner,date_str,winner_str,cage,all_animals
0,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4,"(4.3 (3), 4.4 (4))"
1,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.4 (2))"
2,2022-05-02,4.4 (4) v 3.1 (1),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.1 (1))"
3,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.4 (2))"
4,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4,"(4.3 (3), 4.4 (4))"
5,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4,"(4.3 (3), 4.4 (4))"
6,2022-05-02,4.4 (4) v 3.4 (2),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.4 (2))"
7,2022-05-02,4.3 (3) v 4.4 (4),4.4 (4),2022-05-02,4.4 (4),4,"(4.3 (3), 4.4 (4))"
8,2022-05-02,4.4 (4) v 3.1 (1),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.1 (1))"
9,2022-05-02,4.4 (4) v 3.1 (1),4.4 (4),2022-05-02,4.4 (4),4,"(4.4 (4), 3.1 (1))"


## Get the Elo score for one fight

In [38]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))

# Keeping track of the number of matches
id_to_match_number = defaultdict(lambda:1)
for index, row in home_cage_observations_df.iterrows():
    # Checking if there is an Nan or not
    if row["Winner "] == "nan":
        continue
    else:            
        winner_id = row["winner_str"]
    date = row["date_str"]
    # Getting the ID of the loser subject
    loser_id = list(set(row["all_animals"]) - set([winner_id]))
    loser_id = loser_id[0]
    # Getting the current Elo Score
    current_winner_rating = id_to_elo_score[winner_id] 
    current_loser_rating = id_to_elo_score[loser_id] 
    # Calculating Elo score            
    id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=1, number_of_decimals=1)
    id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=0, number_of_decimals=1)

    # Saving all the data for the winner
    winner_index = next(all_indexes)
    index_to_elo_score_and_meta_data[winner_index]["date"] = date
    index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[date + winner_id]
    index_to_elo_score_and_meta_data[winner_index]["cage"] = row["cage"]
    index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
    index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
    index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
    index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
    index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = 1


    # Saving all the data for the loser
    loser_index = next(all_indexes)
    index_to_elo_score_and_meta_data[loser_index]["date"] = date
    index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[date + loser_id]
    index_to_elo_score_and_meta_data[loser_index]["cage"] = row["cage"]
    index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
    index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
    index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
    index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
    index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = 0

    id_to_match_number[date + winner_id] += 1
    id_to_match_number[date + loser_id] += 1
    

In [39]:
id_to_elo_score

defaultdict(<function __main__.<lambda>()>,
            {'4.4 (4)': 1290.3,
             '4.3 (3)': 924.6,
             '3.4 (2)': 866.3,
             '3.1 (1)': 918.8,
             '4.1 (1)': 1185.0,
             '3.3 (4)': 944.7,
             '3.2 (2)': 889.6,
             '4.2 (3)': 980.7})

In [40]:
index_to_elo_score_and_meta_data[0]

{'date': '2022-05-02',
 'match_number': 1,
 'cage': '4',
 'subject_id': '4.4 (4)',
 'agent_id': '4.3 (3)',
 'original_elo_score': 1000,
 'updated_elo_score': 1010.0,
 'win_draw_loss': 1}

In [41]:
home_cage_elo_score_df = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

In [42]:
home_cage_elo_score_df

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,2022-05-02,1,4,4.4 (4),4.3 (3),1000.0,1010.0,1
1,2022-05-02,1,4,4.3 (3),4.4 (4),1000.0,990.0,0
2,2022-05-02,2,4,4.4 (4),3.4 (2),1010.0,1019.7,1
3,2022-05-02,1,4,3.4 (2),4.4 (4),1000.0,990.3,0
4,2022-05-02,3,4,4.4 (4),3.1 (1),1019.7,1029.1,1
...,...,...,...,...,...,...,...,...
243,2022-05-13,4,4,3.4 (2),4.4 (4),869.6,867.9,0
244,2022-05-13,16,4,4.4 (4),4.3 (3),1286.5,1288.7,1
245,2022-05-13,4,4,4.3 (3),4.4 (4),926.8,924.6,0
246,2022-05-13,17,4,4.4 (4),3.4 (2),1288.7,1290.3,1


In [43]:
home_cage_elo_score_df[home_cage_elo_score_df["date"] == "2022-05-04"]

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
46,2022-05-04,1,3,4.1 (1),3.3 (4),1000.0,1010.0,1
47,2022-05-04,1,3,3.3 (4),4.1 (1),1000.0,990.0,0
48,2022-05-04,2,3,4.1 (1),3.3 (4),1010.0,1019.4,1
49,2022-05-04,2,3,3.3 (4),4.1 (1),990.0,980.6,0
50,2022-05-04,3,3,4.1 (1),3.2 (2),1019.4,1028.8,1
51,2022-05-04,1,3,3.2 (2),4.1 (1),1000.0,990.6,0
52,2022-05-04,4,3,4.1 (1),3.3 (4),1028.8,1037.4,1
53,2022-05-04,3,3,3.3 (4),4.1 (1),980.6,972.0,0
54,2022-05-04,5,3,4.1 (1),3.2 (2),1037.4,1046.1,1
55,2022-05-04,2,3,3.2 (2),4.1 (1),990.6,981.9,0


In [44]:
home_cage_elo_score_df["date"]

0      2022-05-02
1      2022-05-02
2      2022-05-02
3      2022-05-02
4      2022-05-02
          ...    
243    2022-05-13
244    2022-05-13
245    2022-05-13
246    2022-05-13
247    2022-05-13
Name: date, Length: 248, dtype: object

In [45]:
home_cage_elo_score_df[home_cage_elo_score_df["date"] == "2022-05-02" ]

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,2022-05-02,1,4,4.4 (4),4.3 (3),1000.0,1010.0,1
1,2022-05-02,1,4,4.3 (3),4.4 (4),1000.0,990.0,0
2,2022-05-02,2,4,4.4 (4),3.4 (2),1010.0,1019.7,1
3,2022-05-02,1,4,3.4 (2),4.4 (4),1000.0,990.3,0
4,2022-05-02,3,4,4.4 (4),3.1 (1),1019.7,1029.1,1
5,2022-05-02,1,4,3.1 (1),4.4 (4),1000.0,990.6,0
6,2022-05-02,4,4,4.4 (4),3.4 (2),1029.1,1038.0,1
7,2022-05-02,2,4,3.4 (2),4.4 (4),990.3,981.4,0
8,2022-05-02,5,4,4.4 (4),4.3 (3),1038.0,1046.6,1
9,2022-05-02,2,4,4.3 (3),4.4 (4),990.0,981.4,0


In [46]:
home_cage_elo_score_df.head(n=30)

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,2022-05-02,1,4,4.4 (4),4.3 (3),1000.0,1010.0,1
1,2022-05-02,1,4,4.3 (3),4.4 (4),1000.0,990.0,0
2,2022-05-02,2,4,4.4 (4),3.4 (2),1010.0,1019.7,1
3,2022-05-02,1,4,3.4 (2),4.4 (4),1000.0,990.3,0
4,2022-05-02,3,4,4.4 (4),3.1 (1),1019.7,1029.1,1
5,2022-05-02,1,4,3.1 (1),4.4 (4),1000.0,990.6,0
6,2022-05-02,4,4,4.4 (4),3.4 (2),1029.1,1038.0,1
7,2022-05-02,2,4,3.4 (2),4.4 (4),990.3,981.4,0
8,2022-05-02,5,4,4.4 (4),4.3 (3),1038.0,1046.6,1
9,2022-05-02,2,4,4.3 (3),4.4 (4),990.0,981.4,0


In [47]:
home_cage_elo_score_df.tail()

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
243,2022-05-13,4,4,3.4 (2),4.4 (4),869.6,867.9,0
244,2022-05-13,16,4,4.4 (4),4.3 (3),1286.5,1288.7,1
245,2022-05-13,4,4,4.3 (3),4.4 (4),926.8,924.6,0
246,2022-05-13,17,4,4.4 (4),3.4 (2),1288.7,1290.3,1
247,2022-05-13,5,4,3.4 (2),4.4 (4),867.9,866.3,0


In [48]:
home_cage_elo_score_df.to_csv("./proc/home_cage_elo_score.csv")
# elo_score_df.to_excel("./proc/id_to_date_elo_score.xlsx")

# Calculate Elo score for Reward Competition

In [49]:
reward_competition_df = pd.read_excel("../../data/Pilot of Pilot - Reward Competition Video Scoring Assignments.xlsx", sheet_name="Master Master")

In [50]:
[col for col in reward_competition_df.columns if "Time" not in col]

['Date ',
 'Cage',
 'Box',
 'Match',
 'Trial 1 Winner',
 'Trial 2 Winner',
 'Trial 3 Winner',
 'Trial 4 Winner',
 'Trial 5 Winner',
 'Trial 6 Winner',
 'Trial 7 Winner',
 'Trial 8 Winner',
 'Trial 9 Winner',
 'Trial 10 Winner',
 'Trial 11 Winner',
 'Trial 12 Winner',
 'Trial 13 Winner',
 'Trial 14 Winner',
 'Trial 15 Winner',
 'Trial 16 Winner',
 'Trial 17 Winner',
 'Trial 18 Winner',
 'Trial 19 Winner',
 'Trial 20 Winner',
 'Mouse 1 Wins',
 'Mouse 2 Wins',
 'Ties ']

In [51]:
columns_to_keep = ['Date ',
 'Cage',
 'Box',
 'Match',
 'Trial 1 Winner',
 'Trial 2 Winner',
 'Trial 3 Winner',
 'Trial 4 Winner',
 'Trial 5 Winner',
 'Trial 6 Winner',
 'Trial 7 Winner',
 'Trial 8 Winner',
 'Trial 9 Winner',
 'Trial 10 Winner',
 'Trial 11 Winner',
 'Trial 12 Winner',
 'Trial 13 Winner',
 'Trial 14 Winner',
 'Trial 15 Winner',
 'Trial 16 Winner',
 'Trial 17 Winner',
 'Trial 18 Winner',
 'Trial 19 Winner',
 'Trial 20 Winner']

In [52]:
trimmed_reward_competition_df = reward_competition_df[columns_to_keep].copy()

In [53]:
trimmed_reward_competition_df.head()

,Date,Cage,Box,Match,Trial 1 Winner,Trial 2 Winner,Trial 3 Winner,Trial 4 Winner,Trial 5 Winner,Trial 6 Winner,...,Trial 11 Winner,Trial 12 Winner,Trial 13 Winner,Trial 14 Winner,Trial 15 Winner,Trial 16 Winner,Trial 17 Winner,Trial 18 Winner,Trial 19 Winner,Trial 20 Winner
0,2022-05-19,1,1,1.1 v 2.2,1.1,1.1,1.1,1.1,2.2,2.2,...,1.1,2.2,2.2,2.2,2.2,2.2,1.1,2.2,1.1,1.1
1,2022-05-19,1,2,2.3 v 1.4,1.4,1.4,2.3,2.3,1.4,1.4,...,2.3,2.3,2.3,2.3,1.4,1.4,1.4,1.4,2.3,2.3
2,2022-05-19,1,3,1.1 v 2.3,2.3,tie,2.3,1.1,1.1,2.3,...,2.3,2.3,1.1,2.3,2.3,2.3,2.3,2.3,2.3,NaN
3,2022-05-19,1,4,2.2 v 1.4,1.4,2.2,2.2,2.2,1.4,1.4,...,2.2,1.4,2.2,2.2,1.4,2.2,2.2,1.4,1.4,1.4
4,2022-05-19,1,2,2.2 v 2.3,2.2,2.3,2.3,2.3,2.2,2.3,...,2.3,tie,2.2,2.3,2.2,2.2,2.3,2.2,2.2,2.2


In [54]:
trimmed_reward_competition_df = trimmed_reward_competition_df.reset_index()

## Make a Column for each trial

In [55]:
trimmed_reward_competition_df["date_str"] = trimmed_reward_competition_df["Date "].apply(lambda x: str(x.date()))

In [56]:
trimmed_reward_competition_df.columns

Index(['index', 'Date ', 'Cage', 'Box', 'Match', 'Trial 1 Winner',
       'Trial 2 Winner', 'Trial 3 Winner', 'Trial 4 Winner', 'Trial 5 Winner',
       'Trial 6 Winner', 'Trial 7 Winner', 'Trial 8 Winner', 'Trial 9 Winner',
       'Trial 10 Winner', 'Trial 11 Winner', 'Trial 12 Winner',
       'Trial 13 Winner', 'Trial 14 Winner', 'Trial 15 Winner',
       'Trial 16 Winner', 'Trial 17 Winner', 'Trial 18 Winner',
       'Trial 19 Winner', 'Trial 20 Winner', 'date_str'],
      dtype='object')

In [57]:
trimmed_reward_competition_df

,index,Date,Cage,Box,Match,Trial 1 Winner,Trial 2 Winner,Trial 3 Winner,Trial 4 Winner,Trial 5 Winner,...,Trial 12 Winner,Trial 13 Winner,Trial 14 Winner,Trial 15 Winner,Trial 16 Winner,Trial 17 Winner,Trial 18 Winner,Trial 19 Winner,Trial 20 Winner,date_str
0,0,2022-05-19,1,1,1.1 v 2.2,1.1,1.1,1.1,1.1,2.2,...,2.2,2.2,2.2,2.2,2.2,1.1,2.2,1.1,1.1,2022-05-19
1,1,2022-05-19,1,2,2.3 v 1.4,1.4,1.4,2.3,2.3,1.4,...,2.3,2.3,2.3,1.4,1.4,1.4,1.4,2.3,2.3,2022-05-19
2,2,2022-05-19,1,3,1.1 v 2.3,2.3,tie,2.3,1.1,1.1,...,2.3,1.1,2.3,2.3,2.3,2.3,2.3,2.3,NaN,2022-05-19
3,3,2022-05-19,1,4,2.2 v 1.4,1.4,2.2,2.2,2.2,1.4,...,1.4,2.2,2.2,1.4,2.2,2.2,1.4,1.4,1.4,2022-05-19
4,4,2022-05-19,1,2,2.2 v 2.3,2.2,2.3,2.3,2.3,2.2,...,tie,2.2,2.3,2.2,2.2,2.3,2.2,2.2,2.2,2022-05-19
5,5,2022-05-19,1,1,1.1 v 1.4,1.4,1.1,1.4,1.4,1.1,...,1.1,1.1,1.4,1.4,1.4,1.4,1.4,1.4,1.1,2022-05-19
6,6,2022-05-20,1,4,1.1 v 1.4,1.1,1.1,1.1,1.1,1.1,...,1.1,1.1,1.1,1.1,1.1,1.1,Tie,1.4,NO TRIAL,2022-05-20
7,7,2022-05-20,1,3,2.2 v 2.3,2.2,2.2,2.2,2.2,tie,...,2.2,2.2,2.2,2.3,2.3,2.3,2.2,2.3,NaN,2022-05-20
8,8,2022-05-20,1,2,1.1 v 2.3,1.1,tie,2.3,1.1,1.1,...,tie,1.1,2.3,2.3,1.1,1.1,1.1,1.1,1.1,2022-05-20
9,9,2022-05-20,1,1,2.2 v 1.4,2.2,1.4,1.4,1.4,1.2,...,1.4,1.4,1.4,2.2,2.2,2.2,2.2,2.2,NaN,2022-05-20


In [58]:
melted_reward_competition_df = trimmed_reward_competition_df.melt(id_vars=["index", "Date ", "date_str", "Cage", "Box", "Match"], 
        var_name="Trial", 
        value_name="winner")

In [59]:
melted_reward_competition_df.head()

,index,Date,date_str,Cage,Box,Match,Trial,winner
0,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 1 Winner,1.1
1,1,2022-05-19,2022-05-19,1,2,2.3 v 1.4,Trial 1 Winner,1.4
2,2,2022-05-19,2022-05-19,1,3,1.1 v 2.3,Trial 1 Winner,2.3
3,3,2022-05-19,2022-05-19,1,4,2.2 v 1.4,Trial 1 Winner,1.4
4,4,2022-05-19,2022-05-19,1,2,2.2 v 2.3,Trial 1 Winner,2.2


In [60]:
melted_reward_competition_df["Trial Number"] = melted_reward_competition_df["Trial"].apply(lambda x: int(re.search(r'\d+', x).group())
)

In [61]:

reordered_reward_competition_df = melted_reward_competition_df.sort_values(['index', 'Trial Number'], ascending=[True, True]).reset_index(drop=True)

In [62]:
reordered_reward_competition_df

,index,Date,date_str,Cage,Box,Match,Trial,winner,Trial Number
0,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 1 Winner,1.1,1
1,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 2 Winner,1.1,2
2,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 3 Winner,1.1,3
3,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 4 Winner,1.1,4
4,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 5 Winner,2.2,5
...,...,...,...,...,...,...,...,...,...
955,47,2022-05-20,2022-05-20,4,1,4.3 (3) v 4.4 (4),Trial 16 Winner,4.3 (3),16
956,47,2022-05-20,2022-05-20,4,1,4.3 (3) v 4.4 (4),Trial 17 Winner,4.4 (4),17
957,47,2022-05-20,2022-05-20,4,1,4.3 (3) v 4.4 (4),Trial 18 Winner,Tie,18
958,47,2022-05-20,2022-05-20,4,1,4.3 (3) v 4.4 (4),Trial 19 Winner,4.4 (4),19


In [63]:
reordered_reward_competition_df["winner_str"] = reordered_reward_competition_df["winner"].apply(lambda x: str(x).lower().strip())

In [64]:
reordered_reward_competition_df["winner_str"].unique()

array(['1.1', '2.2', '1.4', '2.3', 'tie', 'nan', 'no trial', '1.2', '2.1',
       '1.3', '2.4', '4.1 (1)', '3.2 (2)', '4.2 (3)', '3.3 (4)', '4.1(1)',
       '3.1 (1)', '3.4 (2)', '4.3 (3)', '4.4 (4)'], dtype=object)

# Calculate Elo score for all cells

- Getting all the ID's of the animals

In [65]:
# Splitting by "v" and stripping all the spaces
reordered_reward_competition_df["all_animals"] = reordered_reward_competition_df["Match"].apply(lambda x: tuple([animal.strip() for animal in x.split("v")]))

In [66]:
reordered_reward_competition_df.head()

,index,Date,date_str,Cage,Box,Match,Trial,winner,Trial Number,winner_str,all_animals
0,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 1 Winner,1.1,1,1.1,"(1.1, 2.2)"
1,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 2 Winner,1.1,2,1.1,"(1.1, 2.2)"
2,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 3 Winner,1.1,3,1.1,"(1.1, 2.2)"
3,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 4 Winner,1.1,4,1.1,"(1.1, 2.2)"
4,0,2022-05-19,2022-05-19,1,1,1.1 v 2.2,Trial 5 Winner,2.2,5,2.2,"(1.1, 2.2)"


In [67]:
reordered_reward_competition_df.columns

Index(['index', 'Date ', 'date_str', 'Cage', 'Box', 'Match', 'Trial', 'winner',
       'Trial Number', 'winner_str', 'all_animals'],
      dtype='object')

## Get the Elo score for one fight

In [68]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))

# Keeping track of the number of matches
id_to_match_number = defaultdict(lambda:1)
for index, row in reordered_reward_competition_df.iterrows():
    # Checking if there is an Nan or not
    if row["winner_str"] == "nan":
        continue
    elif row["winner_str"] == "no trial":
        continue
    elif row["winner_str"] == "tie":
        winner_id = row["all_animals"][0]
        loser_id = row["all_animals"][1]
        winner_score = 0.5
        loser_score = 0.5 
    
    
    else:            
        winner_id = row["winner_str"]
        # Getting the ID of the loser subject
        loser_id = list(set(row["all_animals"]) - set([winner_id]))
        loser_id = loser_id[0]
        winner_score = 1
        loser_score = 0
        
        
    # Getting the current Elo Score
    current_winner_rating = id_to_elo_score[winner_id] 
    current_loser_rating = id_to_elo_score[loser_id] 
    # Calculating Elo score            
    id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=winner_score, number_of_decimals=1)
    id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=loser_score, number_of_decimals=1)

    date = row["date_str"]
    # Saving all the data for the winner
    winner_index = next(all_indexes)
    index_to_elo_score_and_meta_data[winner_index]["date"] = date
    index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[date + str(winner_id)]
    index_to_elo_score_and_meta_data[winner_index]["cage"] = row["Cage"]
    index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
    index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
    index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
    index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
    index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = winner_score


    # Saving all the data for the loser
    loser_index = next(all_indexes)
    index_to_elo_score_and_meta_data[loser_index]["date"] = date
    index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[date + str(loser_id)]
    index_to_elo_score_and_meta_data[loser_index]["cage"] = row["Cage"]
    index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
    index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
    index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
    index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
    index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = loser_score

    id_to_match_number[date + str(winner_id)] += 1
    id_to_match_number[date + str(loser_id)] += 1
    

In [69]:
id_to_elo_score

defaultdict(<function __main__.<lambda>()>,
            {'1.1': 1083.3,
             '2.2': 1056.1,
             '1.4': 934.8,
             '2.3': 901.7,
             '1.2': 934.6,
             '2.1': 1067.1,
             '1.3': 976.6,
             '2.4': 1045.8,
             '4.1 (1)': 1072.0,
             '3.2 (2)': 974.5,
             '4.2 (3)': 981.8,
             '3.3 (4)': 964.3,
             '4.1(1)': 1007.4,
             '3.1 (1)': 1125.5,
             '3.4 (2)': 860.5,
             '4.3 (3)': 961.7,
             '4.4 (4)': 1052.3})

In [70]:
index_to_elo_score_and_meta_data[0]

{'date': '2022-05-19',
 'match_number': 1,
 'cage': 1,
 'subject_id': '1.1',
 'agent_id': '2.2',
 'original_elo_score': 1000,
 'updated_elo_score': 1010.0,
 'win_draw_loss': 1}

In [71]:
reward_competition_elo_score_df = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

In [72]:
reward_competition_elo_score_df.head(n=25)

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,2022-05-19,1,1,1.1,2.2,1000.0,1010.0,1.0
1,2022-05-19,1,1,2.2,1.1,1000.0,990.0,0.0
2,2022-05-19,2,1,1.1,2.2,1010.0,1019.4,1.0
3,2022-05-19,2,1,2.2,1.1,990.0,980.6,0.0
4,2022-05-19,3,1,1.1,2.2,1019.4,1028.3,1.0
5,2022-05-19,3,1,2.2,1.1,980.6,971.7,0.0
6,2022-05-19,4,1,1.1,2.2,1028.3,1036.7,1.0
7,2022-05-19,4,1,2.2,1.1,971.7,963.3,0.0
8,2022-05-19,5,1,2.2,1.1,963.3,975.4,1.0
9,2022-05-19,5,1,1.1,2.2,1036.7,1024.6,0.0


In [73]:
reward_competition_elo_score_df[75:125]

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
75,2022-05-19,18,1,2.3,1.4,1006.6,996.2,0.0
76,2022-05-19,19,1,2.3,1.4,996.2,1006.4,1.0
77,2022-05-19,19,1,1.4,2.3,1003.8,993.6,0.0
78,2022-05-19,20,1,2.3,1.4,1006.4,1016.0,1.0
79,2022-05-19,20,1,1.4,2.3,993.6,984.0,0.0
80,2022-05-19,21,1,2.3,1.1,1016.0,1025.7,1.0
81,2022-05-19,21,1,1.1,2.3,1005.9,996.2,0.0
82,2022-05-19,22,1,1.1,2.3,996.2,997.0,0.5
83,2022-05-19,22,1,2.3,1.1,1025.7,1024.9,0.5
84,2022-05-19,23,1,2.3,1.1,1024.9,1034.1,1.0


In [74]:
reward_competition_elo_score_df.to_csv("./proc/reward_competition_elo_score.csv")
# elo_score_df.to_excel("./proc/id_to_date_elo_score.xlsx")

# Calculate Elo Score Urine Marking Assay

In [75]:
urine_marking_assay_df = pd.read_excel("../../data/Pilot of Pilot Consolidation.xlsx", sheet_name="Urine Marking Assay")

In [76]:
urine_marking_assay_df["date_str"] = urine_marking_assay_df["Date"].astype(str)

In [77]:
urine_marking_assay_df["winner_str"] = urine_marking_assay_df["Winner"].astype(str).apply(lambda x: x.strip())

In [78]:
urine_marking_assay_df.head()

,Date,CAGE,Match,Winner,date_str,winner_str
0,2022-05-18,3,4.1 (1) v 3.2 (2),3.2 (2),2022-05-18,3.2 (2)
1,2022-05-18,3,4.2 (3) v 3.3 (4),4.2 (3),2022-05-18,4.2 (3)
2,2022-05-18,4,3.1 (1) v 3.4 (2),3.1 (1),2022-05-18,3.1 (1)
3,2022-05-18,4,4.3 (3) v 4.4 (4),4.4 (4),2022-05-18,4.4 (4)
4,2022-05-19,3,4.1 (1) v 3.2 (2),4.1 (1),2022-05-19,4.1 (1)


# Calculate Elo score for all cells

- Getting all the ID's of the animals

In [79]:
# Splitting by "v" and stripping all the spaces
urine_marking_assay_df["all_animals"] = urine_marking_assay_df["Match"].apply(lambda x: tuple([animal.strip() for animal in x.split("v")]))

In [80]:
urine_marking_assay_df.head(n=25)

,Date,CAGE,Match,Winner,date_str,winner_str,all_animals
0,2022-05-18,3,4.1 (1) v 3.2 (2),3.2 (2),2022-05-18,3.2 (2),"(4.1 (1), 3.2 (2))"
1,2022-05-18,3,4.2 (3) v 3.3 (4),4.2 (3),2022-05-18,4.2 (3),"(4.2 (3), 3.3 (4))"
2,2022-05-18,4,3.1 (1) v 3.4 (2),3.1 (1),2022-05-18,3.1 (1),"(3.1 (1), 3.4 (2))"
3,2022-05-18,4,4.3 (3) v 4.4 (4),4.4 (4),2022-05-18,4.4 (4),"(4.3 (3), 4.4 (4))"
4,2022-05-19,3,4.1 (1) v 3.2 (2),4.1 (1),2022-05-19,4.1 (1),"(4.1 (1), 3.2 (2))"
5,2022-05-19,3,4.2 (3) v 3.3 (4),tie,2022-05-19,tie,"(4.2 (3), 3.3 (4))"
6,2022-05-19,4,3.1 (1) v 3.4 (2),3.4 (2),2022-05-19,3.4 (2),"(3.1 (1), 3.4 (2))"
7,2022-05-19,4,4.3 (3) v 4.4 (4),4.4 (4),2022-05-19,4.4 (4),"(4.3 (3), 4.4 (4))"
8,2022-05-19,1,1.1 v 2.3,1.1,2022-05-19,1.1,"(1.1, 2.3)"
9,2022-05-19,1,2.2 v 1.4,1.4,2022-05-19,1.4,"(2.2, 1.4)"


## Get the Elo score for one fight

In [81]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))

# Keeping track of the number of matches
id_to_match_number = defaultdict(lambda:1)
for index, row in reordered_reward_competition_df.iterrows():
    # Checking if there is an Nan or not
    if row["winner_str"] == "nan":
        continue
    elif row["winner_str"] == "no trial":
        continue
    elif row["winner_str"] == "tie":
        winner_id = row["all_animals"][0]
        loser_id = row["all_animals"][1]
        winner_score = 0.5
        loser_score = 0.5 
    
    
    else:            
        winner_id = row["winner_str"]
        # Getting the ID of the loser subject
        loser_id = list(set(row["all_animals"]) - set([winner_id]))
        loser_id = loser_id[0]
        winner_score = 1
        loser_score = 0
        
        
    # Getting the current Elo Score
    current_winner_rating = id_to_elo_score[winner_id] 
    current_loser_rating = id_to_elo_score[loser_id] 
    # Calculating Elo score            
    id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=winner_score, number_of_decimals=1)
    id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=loser_score, number_of_decimals=1)

    date = row["date_str"]
    # Saving all the data for the winner
    winner_index = next(all_indexes)
    index_to_elo_score_and_meta_data[winner_index]["date"] = date
    index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[date + str(winner_id)]
    index_to_elo_score_and_meta_data[winner_index]["cage"] = row["Cage"]
    index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
    index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
    index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
    index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
    index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = winner_score


    # Saving all the data for the loser
    loser_index = next(all_indexes)
    index_to_elo_score_and_meta_data[loser_index]["date"] = date
    index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[date + str(loser_id)]
    index_to_elo_score_and_meta_data[loser_index]["cage"] = row["Cage"]
    index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
    index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
    index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
    index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
    index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = loser_score

    id_to_match_number[date + str(winner_id)] += 1
    id_to_match_number[date + str(loser_id)] += 1
    

In [82]:
# Dictionary that keeps track of the current Elo score of the subject
id_to_elo_score = defaultdict(lambda:1000)

index_to_elo_score_and_meta_data = defaultdict(dict)
# Iterating through each column which is a day
all_indexes = iter(range(0, 99999))

# Keeping track of the number of matches
id_to_match_number = defaultdict(lambda:1)
for index, row in urine_marking_assay_df.iterrows():
    # Checking if there is an Nan or not
    if row["winner_str"] == "nan":
        continue
    elif row["winner_str"] == "tie":
        winner_id = row["all_animals"][0]
        loser_id = row["all_animals"][1]
        winner_score = 0.5
        loser_score = 0.5  
    else:            
        winner_id = row["winner_str"]
        # Getting the ID of the loser subject
        loser_id = list(set(row["all_animals"]) - set([winner_id]))
        loser_id = loser_id[0]
        winner_score = 1
        loser_score = 0
        
        
        
        
    date = row["date_str"]
    # Getting the ID of the loser subject
    loser_id = list(set(row["all_animals"]) - set([winner_id]))
    loser_id = loser_id[0]
    # Getting the current Elo Score
    current_winner_rating = id_to_elo_score[winner_id] 
    current_loser_rating = id_to_elo_score[loser_id] 
    # Calculating Elo score            
    id_to_elo_score[winner_id] = calculate_elo_score(subject_elo_score=current_winner_rating, agent_elo_score=current_loser_rating, score=winner_score, number_of_decimals=1)
    id_to_elo_score[loser_id] = calculate_elo_score(subject_elo_score=current_loser_rating, agent_elo_score=current_winner_rating, score=loser_score, number_of_decimals=1)

    # Saving all the data for the winner
    winner_index = next(all_indexes)
    index_to_elo_score_and_meta_data[winner_index]["date"] = date
    index_to_elo_score_and_meta_data[winner_index]["match_number"] = id_to_match_number[date + winner_id]
    index_to_elo_score_and_meta_data[winner_index]["cage"] = row["CAGE"]
    index_to_elo_score_and_meta_data[winner_index]["subject_id"] = winner_id
    index_to_elo_score_and_meta_data[winner_index]["agent_id"] = loser_id
    index_to_elo_score_and_meta_data[winner_index]["original_elo_score"] = current_winner_rating
    index_to_elo_score_and_meta_data[winner_index]["updated_elo_score"] = id_to_elo_score[winner_id]
    index_to_elo_score_and_meta_data[winner_index]["win_draw_loss"] = winner_score


    # Saving all the data for the loser
    loser_index = next(all_indexes)
    index_to_elo_score_and_meta_data[loser_index]["date"] = date
    index_to_elo_score_and_meta_data[loser_index]["match_number"] = id_to_match_number[date + loser_id]
    index_to_elo_score_and_meta_data[loser_index]["cage"] = row["CAGE"]
    index_to_elo_score_and_meta_data[loser_index]["subject_id"] = loser_id
    index_to_elo_score_and_meta_data[loser_index]["agent_id"] = winner_id
    index_to_elo_score_and_meta_data[loser_index]["original_elo_score"] = current_loser_rating
    index_to_elo_score_and_meta_data[loser_index]["updated_elo_score"] = id_to_elo_score[loser_id]
    index_to_elo_score_and_meta_data[loser_index]["win_draw_loss"] = loser_score

    id_to_match_number[date + winner_id] += 1
    id_to_match_number[date + loser_id] += 1
    

In [83]:
id_to_elo_score

defaultdict(<function __main__.<lambda>()>,
            {'3.2 (2)': 999.4,
             '4.1 (1)': 1020.6,
             '4.2 (3)': 978.9,
             '3.3 (4)': 1001.1,
             '3.1 (1)': 1009.0,
             '3.4 (2)': 971.0,
             '4.4 (4)': 1054.6,
             '4.3 (3)': 965.4,
             '1.1': 1028.3,
             '2.3': 963.4,
             '1.4': 1007.2,
             '2.2': 1001.1,
             '2.4': 1000.6,
             '1.2': 962.8,
             '2.1': 1037.2,
             '1.3': 999.4})

In [84]:
index_to_elo_score_and_meta_data[0]

{'date': '2022-05-18',
 'match_number': 1,
 'cage': 3,
 'subject_id': '3.2 (2)',
 'agent_id': '4.1 (1)',
 'original_elo_score': 1000,
 'updated_elo_score': 1010.0,
 'win_draw_loss': 1}

In [85]:
urine_marking_elo_score_df = pd.DataFrame.from_dict(index_to_elo_score_and_meta_data, orient="index")

In [88]:
urine_marking_elo_score_df.head(n=25)

,date,match_number,cage,subject_id,agent_id,original_elo_score,updated_elo_score,win_draw_loss
0,2022-05-18,1,3,3.2 (2),4.1 (1),1000.0,1010.0,1.0
1,2022-05-18,1,3,4.1 (1),3.2 (2),1000.0,990.0,0.0
2,2022-05-18,1,3,4.2 (3),3.3 (4),1000.0,1010.0,1.0
3,2022-05-18,1,3,3.3 (4),4.2 (3),1000.0,990.0,0.0
4,2022-05-18,1,4,3.1 (1),3.4 (2),1000.0,1010.0,1.0
5,2022-05-18,1,4,3.4 (2),3.1 (1),1000.0,990.0,0.0
6,2022-05-18,1,4,4.4 (4),4.3 (3),1000.0,1010.0,1.0
7,2022-05-18,1,4,4.3 (3),4.4 (4),1000.0,990.0,0.0
8,2022-05-19,1,3,4.1 (1),3.2 (2),990.0,1000.6,1.0
9,2022-05-19,1,3,3.2 (2),4.1 (1),1010.0,999.4,0.0


In [90]:
home_cage_elo_score_df.to_csv("./proc/urine_marking_elo_score.csv")